<a href="https://colab.research.google.com/github/shravan1994/Talking-data-User-demographic-prediction/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Our Approach 

<p>We did not get good accuracy with using single model to predict the user group</p>
<p>
In this notebook we tried another approach where we are trying to predict the gender first and using that predicted gender as a new features to predict the user demographic group.</p>

<p>
Also we trained seperate models for devices having events and the devices which don't have any events data.
</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install xgboost --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Loading the data into memory

In [ ]:
if __name__ == '__main__':
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if __name__ == '__main__':
  import os
  os.environ['KAGGLE_CONFIG_DIR'] = "."

  !kaggle competitions download -c talkingdata-mobile-user-demographics

  !unzip '/content/talkingdata-mobile-user-demographics.zip'
  !unzip '/content/app_events.csv.zip' -d data
  !unzip '/content/app_labels.csv.zip' -d data
  !unzip '/content/events.csv.zip' -d data
  !unzip '/content/gender_age_test.csv.zip' -d data
  !unzip '/content/gender_age_train.csv.zip' -d data
  !unzip '/content/label_categories.csv.zip' -d data
  !unzip '/content/phone_brand_device_model.csv.zip' -d data

 99% 281M/283M [00:05<00:00, 58.6MB/s]
100% 283M/283M [00:05<00:00, 56.9MB/s]
Archive:  /content/talkingdata-mobile-user-demographics.zip
  inflating: app_events.csv.zip      
  inflating: app_labels.csv.zip      
  inflating: events.csv.zip          
  inflating: gender_age_test.csv.zip  
  inflating: gender_age_train.csv.zip  
  inflating: label_categories.csv.zip  
  inflating: phone_brand_device_model.csv.zip  
  inflating: sample_submission.csv.zip  
Archive:  /content/app_events.csv.zip
  inflating: data/app_events.csv     
Archive:  /content/app_labels.csv.zip
  inflating: data/app_labels.csv     
Archive:  /content/events.csv.zip
  inflating: data/events.csv         
Archive:  /content/gender_age_test.csv.zip
  inflating: data/gender_age_test.csv  
Archive:  /content/gender_age_train.csv.zip
  inflating: data/gender_age_train.csv  
Archive:  /content/label_categories.csv.zip
  inflating: data/label_categories.csv  
Archive:  /content/phone_brand_device_model.csv.zip
  inflating

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer

from geopy.distance import geodesic, great_circle
from scipy.sparse import hstack, save_npz
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix

In [ ]:
# drive path to save models
drive_path = '/content/drive/MyDrive/Colab Notebooks/Self case study/Talking Data User Demographics Prediction/models'

## Getting different features, using below methods

### 1. Bag of categories

In [ ]:
def get_bag_of_categories(devices, all_events_df, all_app_events_df, app_labels_df, label_categories_df, training=False):
  import pickle
  filename = f'{drive_path}/categories_vectorizer.sav'
  app_categories_df = app_labels_df.merge(label_categories_df, on='label_id', how='left')
  app_categories_dict = dict(zip(app_categories_df.app_id, app_categories_df.category))
  
  device_list = devices['device_id']
  events = all_events_df[all_events_df['device_id'].isin(device_list)][['event_id', 'device_id']]
  app_events = all_app_events_df[all_app_events_df['event_id'].isin(events['event_id'])]
  app_events = events.merge(app_events, on='event_id', how='right')
  app_events = app_events.groupby('device_id').app_id.apply(lambda x: ' '.join([app_categories_dict.get(app_id) for app_id in x])).to_frame()
  app_events['app_id'] = app_events['app_id'].str.lower()

  app_events = devices.merge(app_events, on='device_id', how='left')
  app_events['app_id'] = app_events['app_id'].fillna('na')

  if training:
    vocabulary = label_categories_df['category'].fillna('unknown').str.lower().unique().tolist() + ['na']
    vectorizer = CountVectorizer(vocabulary=vocabulary)
    categories_bow_vector = vectorizer.fit_transform(app_events['app_id'])
    
    # saving the vectorizer to drive
    pickle.dump(vectorizer, open(filename, 'wb'))
    return categories_bow_vector
  else:
    vectorizer = pickle.load(open(filename, 'rb'))
    categories_bow_vector = vectorizer.transform(app_events['app_id'])
    return categories_bow_vector

### 2. bag of installed app_ids

In [ ]:
def get_bag_of_apps(devices, app_labels_df, all_events_df, all_app_events_df, label_categories_df, training=False):
  import pickle
  filename = f'{drive_path}/apps_vectorizer.sav'

  device_list = devices['device_id']
  app_categories_df = app_labels_df.merge(label_categories_df, on='label_id', how='left')

  events = all_events_df[all_events_df['device_id'].isin(device_list)][['event_id', 'device_id']]
  app_events = all_app_events_df[all_app_events_df['event_id'].isin(events['event_id'])]
  app_events = events.merge(app_events, on='event_id', how='right')
  app_events = app_events.groupby('device_id').app_id.apply(lambda x: ' '.join([str(app_id) for app_id in x])).to_frame()
  
  app_events = devices.merge(app_events, on='device_id', how='left')
  app_events['app_id'] = app_events['app_id'].fillna('na')

  if training:
    vocabulory = list(app_categories_df['app_id'].astype(str).unique()) + ['na']
    vectorizer = CountVectorizer(vocabulary=vocabulory)
    apps_bow_vector = vectorizer.fit_transform(app_events['app_id'])

    # saving the vectorizer to drive
    pickle.dump(vectorizer, open(filename, 'wb'))
    return apps_bow_vector
  else:
    vectorizer = pickle.load(open(filename, 'rb'))
    categories_bow_vector = vectorizer.transform(app_events['app_id'])
    return categories_bow_vector

### 3. phone brand response encoding

In [ ]:
def get_phone_brand_features_response_enc(devices, phone_brand_df, y_train, label=None, training=False):
  import pickle
  import json

  # if we want features for gender prediction 
  # then labels should be only 0, 1
  # and otherwise, it would be 12 labels 0 to 11
  if 'gender' in label:
    labels = np.array([0, 1])
  else:
    labels = np.array(list(range(0,12)))

  filename = f'{drive_path}/response_encoding_phone_brand{label}.pkl'

  phone_brand_df = phone_brand_df.groupby('device_id').agg(
      phone_brand = ('phone_brand', lambda x: ' '.join(np.unique(x)))
  ).reset_index()

  if training:
    encoding_dict = {}
    brand_and_model = devices.merge(phone_brand_df, on='device_id', how='left')
    brand_and_model['label'] = y_train

    for brand_name in brand_and_model['phone_brand'].unique():
      if not brand_name:
        continue

      temp = brand_and_model[brand_and_model['phone_brand'] == brand_name]
      totals = len(temp)
      
      encoding_list = []
      for label in labels:
        count = len(temp[temp['label'] == label])
        encoding_list.append(count/totals)

      encoding_dict.update({
          brand_name: encoding_list
      })
      
    brand_and_model[labels] = brand_and_model['phone_brand'].apply(lambda x: pd.Series(encoding_dict.get(x)))
    brand_and_model = brand_and_model.drop(['phone_brand', 'label'], axis=1)
    brand_and_model = brand_and_model[labels].values
    # save to disk
    json.dump(encoding_dict, open(filename, 'w'))

    return brand_and_model
  else:
    encoding_dict = json.load(open(filename))
    brand_and_model = devices.merge(phone_brand_df, on='device_id', how='left')
    brand_and_model[labels] = brand_and_model['phone_brand'].apply(lambda x: pd.Series(encoding_dict.get(x)))
    brand_and_model = brand_and_model.drop(['phone_brand'], axis=1)
    brand_and_model = brand_and_model[labels].values
    return brand_and_model

### 4. device model response encoding

In [ ]:
def get_device_model_features_response_enc(devices, phone_brand_df, y_train, label=None, training=False):
  import pickle
  import json

  # if we want features for gender prediction 
  # then labels should be only 0, 1
  # and otherwise, it would be 12 labels 0 to 11
  if 'gender' in label:
    labels = np.array([0, 1])
  else:
    labels = np.array(list(range(0,12)))

  filename = f'{drive_path}/response_encoding_device_model_{label}.pkl'

  phone_brand_df = phone_brand_df.groupby('device_id').agg(
    device_model = ('device_model', lambda x: ' '.join(np.unique(x)))
  ).reset_index()

  if training:
    encoding_dict = {}
    brand_and_model = devices.merge(phone_brand_df, on='device_id', how='left')
    brand_and_model['label'] = y_train

    for device_model in brand_and_model['device_model'].unique():
      if not device_model:
        continue

      temp = brand_and_model[brand_and_model['device_model'] == device_model]
      totals = len(temp)
      
      encoding_list = []
      for label in labels:
        count = len(temp[temp['label'] == label])
        encoding_list.append(count/totals)

      encoding_dict.update({
          device_model: encoding_list
      })

    brand_and_model[labels] = brand_and_model['device_model'].apply(lambda x: pd.Series(encoding_dict.get(x)))
    brand_and_model = brand_and_model.drop(['device_model', 'label'], axis=1)
    brand_and_model = brand_and_model[labels].values
    # save to disk
    json.dump(encoding_dict, open(filename, 'w'))
    return brand_and_model
  else:
    encoding_dict = json.load(open(filename))

    brand_and_model = devices.merge(phone_brand_df, on='device_id', how='left')
    brand_and_model[labels] = brand_and_model['device_model'].apply(lambda x: pd.Series(encoding_dict.get(x)))
    brand_and_model = brand_and_model.drop(['device_model'], axis=1)
    brand_and_model = brand_and_model[labels].values
    return brand_and_model

### 5. Bag of Phone brands

In [ ]:
def get_phone_brand_features_bow(devices, phone_brand_df, training=False):
  import pickle
  filename = f'{drive_path}/bow_phone_brand.pkl'
  vocabulary = phone_brand_df['phone_brand'].str.lower().unique().tolist()

  brand_and_model = devices.merge(phone_brand_df, on='device_id', how='left')
  brand_and_model = brand_and_model.groupby('device_id').phone_brand.apply(lambda x: ' '.join([s for s in x])).to_frame()
  
  if training:
    vectorizer = CountVectorizer(vocabulary=vocabulary)
    bow_vector = vectorizer.fit_transform(brand_and_model['phone_brand'])
    
    # saving the vectorizer to drive
    pickle.dump(vectorizer, open(filename, 'wb'))
    return bow_vector
  else:
    vectorizer = pickle.load(open(filename, 'rb'))
    bow_vector = vectorizer.transform(brand_and_model['phone_brand'])
    return bow_vector

### 6. Bag of device model

In [ ]:
def get_device_model_features_bow(devices, phone_brand_df, training=False):
  import pickle
  filename = f'{drive_path}/bow_device_model.pkl'
  vocabulary = phone_brand_df['device_model'].str.lower().unique().tolist()

  device_model = devices.merge(phone_brand_df, on='device_id', how='left')
  device_model = device_model.groupby('device_id').device_model.apply(lambda x: ' '.join([s for s in x])).to_frame()

  if training:
    vectorizer = CountVectorizer(vocabulary=vocabulary)
    bow_vector = vectorizer.fit_transform(device_model['device_model'])
    # saving the vectorizer to drive
    pickle.dump(vectorizer, open(filename, 'wb'))
    return bow_vector
  else:
    vectorizer = pickle.load(open(filename, 'rb'))
    bow_vector = vectorizer.transform(device_model['device_model'])
    return bow_vector

### 7. Total events

In [ ]:
def get_total_events(devices, all_events_df, training=False):
  events = all_events_df[all_events_df['device_id'].isin(devices['device_id'])]
  events = devices.merge(events, on='device_id', how='left')
  events = events.groupby('device_id').event_id.apply(lambda x: len(x)).to_frame()
  return events['event_id'].values.reshape(-1, 1)

### 8. Total apps

In [ ]:
def total_apps_installed(devices, all_events_df, all_app_events_df, training=False):
  device_list = devices['device_id']
  events = all_events_df[all_events_df['device_id'].isin(device_list)][['event_id', 'device_id']]
  app_events = all_app_events_df[all_app_events_df['event_id'].isin(events['event_id'])]
  app_events = events.merge(app_events, on='event_id', how='right')
  total_apps = app_events.groupby('device_id').agg({'app_id': 'nunique'}).reset_index()
  total_apps = devices.merge(total_apps, on='device_id', how='left')
  total_apps['app_id'] = total_apps.app_id.fillna(0)
  total_apps = total_apps.drop('device_id', axis=1)
  
  return total_apps.values

### 9. distance travelled

In [ ]:
def distance_traveled_per_day(devices, all_events_df, training=False):
    import pickle
    filename = f'{drive_path}/distance_travelled_columns.sav'
    def get_distance_travelled(x):
        x = x.sort_values('datetime')
        locations = x.to_dict('records')
        total_d = 0
        
        for i, _ in enumerate(locations):
            if i == 0:
                continue
            
            # ignoring the invalid lat and long co-ordinates
            if locations[i-1].get('latitude') == 0.0 or locations[i].get('latitude') == 0.0 or locations[i-1].get('latitude') == 1.0 or locations[i].get('latitude') == 1.0:
                continue

            distance = great_circle(
                (locations[i-1].get('latitude'), locations[i-1].get('longitude')), 
                (locations[i].get('latitude'), locations[i].get('longitude'))
            ).km
            total_d += distance
        
        return total_d
    
    device_list = devices['device_id']
    device_events = all_events_df[all_events_df['device_id'].isin(device_list)].copy()
    device_events['datetime'] = pd.to_datetime(device_events['timestamp'])
    device_events['day'] = device_events['datetime'].dt.day

    if training:
      columns = list(device_events['day'].unique())
      
      # saving the vectorizer to drive
      pickle.dump(columns, open(filename, 'wb'))
    else:
      columns = pickle.load(open(filename, 'rb'))

    def get_features(x):
      features = {
          int(i): 0
          for i in columns
      }
      for row in x.iterrows():
        key = int(row[1].day)
        features[key] = row[1].distance

      return pd.Series(list(features.values()))

    # print('distance travelled 1')
    device_events = device_events.groupby(['device_id', 'day'])[['datetime', 'latitude', 'longitude']].apply(
      lambda x: get_distance_travelled(x)
    ).to_frame().reset_index().rename(columns={0: 'distance'})
    # print('distance travelled 2')
    device_events = device_events.groupby('device_id').apply(get_features)
    print('distance travelled ', device_events.shape)
    return device_events.values

In [ ]:
# devices = pd.read_csv('data/gender_age_train.csv')
# distance_travelled = distance_traveled_per_day(devices, all_events_df, training=False)

### 10. Has events

In [ ]:
def has_events(devices, all_events_df):
  import math

  device_has_events = all_events_df.groupby('device_id').agg({'event_id': 'nunique'}).reset_index()
  device_events = devices.merge(device_has_events, on='device_id', how='left')
  device_events['has_events'] = pd.notna(device_events['event_id'])
  device_events['has_events'] = device_events['has_events'].replace({True: 1})
  device_events['has_events'] = device_events['has_events'].replace({False: 0})
  return device_events['has_events'].values    

### 11. hourly events

In [ ]:
def get_hourly_events(devices, all_events_df, training=False):
  import pickle
  filename = f'{drive_path}/hourly_events.pkl'

  events = all_events_df[all_events_df['device_id'].isin(devices['device_id'])]
  events['hour'] = pd.to_datetime(events['timestamp']).dt.hour
  events = events.groupby(['device_id']).hour.apply(lambda x: ' '.join([str(s) for s in x])).to_frame().reset_index()
  events = devices.merge(events, on='device_id', how='left')
  events.hour = events.hour.fillna('na')

  if training:
    vocabulary = [str(h) for h in range(0, 24)] + ['na']
    vectorizer = CountVectorizer(vocabulary=vocabulary)
    bow_vector = vectorizer.fit_transform(events['hour'])
    pickle.dump(vectorizer, open(filename, 'wb'))
    return bow_vector
  else:
    vectorizer = pickle.load(open(filename, 'rb'))
    bow_vector = vectorizer.transform(events['hour'])
    return bow_vector

### 11. median latitude and longitude

In [ ]:
def get_median_lat_long(devices, all_events_df):
  device_list = devices['device_id']
  events = all_events_df[all_events_df['device_id'].isin(device_list)][['device_id', 'latitude', 'longitude']]
  
  events.longitude=events.longitude.apply(lambda x:np.NaN if x==0 else x) 
  events.latitude=events.latitude.apply(lambda x:np.NaN if x==0 else x)

  lat_long = events.groupby('device_id').agg({
      'latitude': 'median',
      'longitude': 'median',
  }).reset_index()
  lat_long = devices.merge(lat_long, on='device_id', how='left')
  lat_long[['latitude', 'longitude']] = lat_long[['latitude', 'longitude']].fillna(0)
  
  return lat_long[['latitude', 'longitude']].values

### Extracting features in training data and save intermediate models to use on test data.

In [ ]:
# all_events_df = pd.read_csv('data/events.csv')
# all_app_events_df = pd.read_csv('data/app_events.csv')
# app_labels_df = pd.read_csv('data/app_labels.csv')
# label_categories_df = pd.read_csv('data/label_categories.csv')
# phone_brand_df = pd.read_csv('data/phone_brand_device_model.csv')

## Get features for training data.
<p>We will also save the counter vectorizers used for multiple features, which we can use while preparing features for test data</p>

In [ ]:
def select_k_best(features, y_train, label, n=1000, training=False):
  from sklearn.feature_selection import SelectKBest, chi2
  import pickle
  filename = f'{drive_path}/select_k_best_{label}.pkl'

  if training:
    kbest = SelectKBest(chi2, k=n)
    best_features = kbest.fit_transform(features, y_train)
    pickle.dump(kbest, open(filename, 'wb'))
    return best_features
  else:
    kbest = pickle.load(open(filename, 'rb'))
    best_features = kbest.transform(features)
    return best_features

In [ ]:
def remove_features_with_no_variance(features, training=False, label='with_events'):
  from sklearn.feature_selection import VarianceThreshold
  import pickle
  filename = f'{drive_path}/no_variance_model_{label}.pkl'
  
  if training:
    selector = VarianceThreshold(threshold=0.0)
    best_features = selector.fit_transform(features)
    pickle.dump(selector, open(filename, 'wb'))
    return best_features
  else:
    selector = pickle.load(open(filename, 'rb'))
    best_features = selector.transform(features)
    return best_features

In [ ]:
def get_features_with_events(train_data_devices, y_train, training=False, label='gender'):
  all_events_df = pd.read_csv('data/events.csv')
  all_app_events_df = pd.read_csv('data/app_events.csv')
  app_labels_df = pd.read_csv('data/app_labels.csv')
  label_categories_df = pd.read_csv('data/label_categories.csv')
  phone_brand_df = pd.read_csv('data/phone_brand_device_model.csv')

  categories_bow = get_bag_of_categories(
    train_data_devices, all_events_df, all_app_events_df, 
    app_labels_df, label_categories_df, training=training
  )
  apps_bow = get_bag_of_apps(train_data_devices, app_labels_df, all_events_df, all_app_events_df, label_categories_df, training=False)
  phone_brand_resp_enc = get_phone_brand_features_response_enc(
      train_data_devices, phone_brand_df, y_train, label=f'{label}_with_events', training=training)
  device_model_resp_enc = get_device_model_features_response_enc(
      train_data_devices, phone_brand_df, y_train, label=f'{label}_with_events', training=training)

  bow_phone_brand = get_phone_brand_features_bow(train_data_devices, phone_brand_df, training=training)
  bow_device_model = get_device_model_features_bow(train_data_devices, phone_brand_df, training=training)
  total_events = get_total_events(train_data_devices, all_events_df, training=training)
  total_apps = total_apps_installed(train_data_devices, all_events_df, all_app_events_df, training=training)
  distance_travelled = distance_traveled_per_day(train_data_devices, all_events_df, training=training)
  median_lat_long = get_median_lat_long(train_data_devices, all_events_df)
  hourly_events = get_hourly_events(train_data_devices, all_events_df, training=training)

  apps_bow_best = select_k_best(apps_bow, y_train, label=label, n=2000, training=training)
  # total_events
  X = hstack([categories_bow, apps_bow_best, 
            bow_phone_brand, bow_device_model, 
            total_apps, distance_travelled, median_lat_long, hourly_events])
  
  print('data shape before removing no variance data: ', X.shape)
  print('training: ', training)
  X = remove_features_with_no_variance(X, training=training, label=f'{label}_with_events')
  print('data shape', X.shape)

  return X

In [ ]:
def get_features_with_no_events(train_data_devices, y_train, training=False, label='gender'):
  phone_brand_df = pd.read_csv('data/phone_brand_device_model.csv')
  bow_phone_brand = get_phone_brand_features_bow(train_data_devices, phone_brand_df, training=training)
  bow_device_model = get_device_model_features_bow(train_data_devices, phone_brand_df, training=training)
  phone_brand_resp_enc = get_phone_brand_features_response_enc(
      train_data_devices, phone_brand_df, y_train, label=f'{label}_no_events', training=training)
  device_model_resp_enc = get_device_model_features_response_enc(
      train_data_devices, phone_brand_df, y_train, label=f'{label}_no_events', training=training)
  X = hstack([bow_phone_brand, bow_device_model, phone_brand_resp_enc,device_model_resp_enc])
  X = remove_features_with_no_variance(X, training=training, label=f'{label}_no_events')
  print('data shape', X.shape)
  
  return X

In [ ]:
def get_labels(y, training=False, label='group'):
  from sklearn.preprocessing import LabelEncoder
  import pickle

  filename = f'{drive_path}/label_encoder_{label}.pkl'

  if training:
    enc = LabelEncoder()
    y =  enc.fit_transform(y)
    pickle.dump(enc, open(filename, 'wb'))
    return y
  else:
    enc = pickle.load(open(filename, 'rb'))
    y =  enc.transform(y)
    return y

## Confusion matrix

In [ ]:
def plot_confusion_matrix(test_y, predict_y, class_labels):
    C = confusion_matrix(test_y, predict_y)
    print("Number of misclassified points ",(len(test_y)-np.trace(C))/len(test_y)*100)
    
    A =(((C.T)/(C.sum(axis=1))).T)
    
    B =(C/C.sum(axis=0))
    print(1)
    labels = [0,1,2,3,4,5,6,7,8,9,10,11]
    cmap=sns.light_palette("green")

    # representing A in heatmap format
    print("-"*50, "Confusion matrix", "-"*50)
    plt.figure(figsize=(10,5))
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    print(2)
    print("-"*50, "Precision matrix", "-"*50)
    plt.figure(figsize=(10,5))
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    print(3)
    # representing B in heatmap format
    print("-"*50, "Recall matrix"    , "-"*50)
    plt.figure(figsize=(10,5))
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    print(4)

## Predicting gender for devices with events

In [ ]:
def train_gender_predictor_with_events(devices_events_gen, y_gen_label, training=False):
  X_train = get_features_with_events(devices_events_gen, y_gen_label, training=training, label='gender')
  if training:
    print('train dataset\'s shape')
    print(X_train.shape)
    print(y_gen_label.shape)

    gpu_dict = {
      'objective': 'binary:logistic',
      'tree_method': 'gpu_hist',
      "eval_metric": "logloss"
    }

    param_grid = {'max_depth': [2, 3, 4], 'n_estimators': [200, 300, 400, 500]}
    xgb = XGBClassifier(**gpu_dict, random_state=42)

    gs = GridSearchCV(
            estimator=xgb,
            param_grid=param_grid, 
            cv=4, 
            n_jobs=-1, 
            scoring='neg_log_loss',
            verbose=10
        )
    gs.fit(X_train, y_gen_label)

    print('Best params ', gs.best_params_)
    x_cfl = gs.best_estimator_
        
    cali_cfl = CalibratedClassifierCV(x_cfl, method="sigmoid")
    cali_cfl.fit(X_train, y_gen_label)
          
    predict_y_train_proba = cali_cfl.predict_proba(X_train)
    predict_y_train = cali_cfl.predict(X_train)
      
    # saving the model to disk for later usage
    import pickle
    file_name = f"{drive_path}/best_model_gender_with_events.pkl"
    pickle.dump(cali_cfl, open(file_name, "wb"))
    return predict_y_train, predict_y_train_proba
  else:
    print('testing')
    import pickle
    file_name = f"{drive_path}/best_model_gender_with_events.pkl"
    cali_cfl = pickle.load(open(file_name, "rb"))
    predict_y_test_proba = cali_cfl.predict_proba(X_train)
    predict_y_test = cali_cfl.predict(X_train)
    return predict_y_test, predict_y_test_proba

## Predicting gender based on devices with no events

In [ ]:
def train_gender_predictor_with_no_events(devices_no_events_gen, y_gen_label, training=False):
  import pickle
  file_name = f"{drive_path}/best_model_gender_with_no_events.pkl"
  X_train = get_features_with_no_events(devices_no_events_gen, y_gen_label, training=training, label='gender')
  if training:
    print('train dataset\'s shape')
    print(X_train.shape)
    print(y_gen_label.shape)

    gpu_dict = {
      'objective': 'binary:logistic',
      'tree_method': 'gpu_hist',
      "eval_metric": "logloss"
    }

    param_grid = {'max_depth': [2, 3, 4], 'n_estimators': [200, 300, 400, 500]}
    xgb = XGBClassifier(**gpu_dict, random_state=42)

    gs = GridSearchCV(
            estimator=xgb,
            param_grid=param_grid, 
            cv=4, 
            n_jobs=-1, 
            scoring='neg_log_loss',
            verbose=10
        )
    gs.fit(X_train, y_gen_label)

    print('Best params ', gs.best_params_)
    x_cfl = gs.best_estimator_
        
    cali_cfl = CalibratedClassifierCV(x_cfl, method="sigmoid")
    cali_cfl.fit(X_train, y_gen_label)
          
    predict_y_train_proba = cali_cfl.predict_proba(X_train)
    predict_y_train = cali_cfl.predict(X_train)
      
    # saving the model to disk for later usage
    pickle.dump(cali_cfl, open(file_name, "wb"))
    return predict_y_train, predict_y_train_proba
  else:
    cali_cfl = pickle.load(open(file_name, "rb"))
    predict_y_test_proba = cali_cfl.predict_proba(X_train)
    predict_y_test = cali_cfl.predict(X_train)
    return predict_y_test, predict_y_test_proba

## Predicting user groups on devices with events

In [ ]:
def train_user_group_with_events(devices_events, gender_pred, y_tr_events, training=False):
  import pickle
  file_name = f"{drive_path}/best_model_group_with_events.pkl"
    
  X_events = get_features_with_events(devices_events, y_tr_events, training=training, label='group')
  X_events = hstack([X_events, gender_pred])
  
  if training:
    gpu_dict = {
      'objective': 'multi:softproba',
      'tree_method': 'gpu_hist',
      "eval_metric": "mlogloss"
    }

    param_grid = {'max_depth': [2, 3, 4], 'n_estimators': [200, 300, 400, 500]}
    xgb = XGBClassifier(**gpu_dict, random_state=42)

    gs = GridSearchCV(
            estimator=xgb,
            param_grid=param_grid, 
            cv=4, 
            n_jobs=-1, 
            scoring='neg_log_loss',
            verbose=10
        )
    gs.fit(X_events, y_tr_events)
    print('Best params ', gs.best_params_)
    x_cfl = gs.best_estimator_
        
    cali_cfl = CalibratedClassifierCV(x_cfl, method="sigmoid")
    cali_cfl.fit(X_events, y_tr_events)
          
    predict_y_proba = cali_cfl.predict_proba(X_events)
    predict_y = cali_cfl.predict(X_events)
    
    # saving the model to disk for later usage
    pickle.dump(cali_cfl, open(file_name, "wb"))
    return predict_y, predict_y_proba
  else:
    # loading the model
    print('testing part')
    cali_cfl = pickle.load(open(file_name, "rb"))
    predict_y_proba = cali_cfl.predict_proba(X_events)
    predict_y = cali_cfl.predict(X_events)
    return predict_y, predict_y_proba

## Predicting user group on devices with no events

In [ ]:
def train_user_group_with_no_events(devices_events, gender_pred, y_tr_events, training=False):
  import pickle
  file_name = f"{drive_path}/best_model_group_with_no_events.pkl"
    
  X_events = get_features_with_no_events(devices_events, y_tr_events, training=training, label='group')
  X_events = hstack([X_events, gender_pred])
  
  if training:
    gpu_dict = {
      'objective': 'multi:softproba',
      'tree_method': 'gpu_hist',
      "eval_metric": "mlogloss"
    }

    param_grid = {'max_depth': [2, 3, 4], 'n_estimators': [200, 300, 400, 500]}
    xgb = XGBClassifier(**gpu_dict, random_state=42)

    gs = GridSearchCV(
            estimator=xgb,
            param_grid=param_grid, 
            cv=4, 
            n_jobs=-1, 
            scoring='neg_log_loss',
            verbose=10
        )
    gs.fit(X_events, y_tr_events)
    print('Best params ', gs.best_params_)
    x_cfl = gs.best_estimator_
        
    cali_cfl = CalibratedClassifierCV(x_cfl, method="sigmoid")
    cali_cfl.fit(X_events, y_tr_events)
          
    predict_y_proba = cali_cfl.predict_proba(X_events)
    predict_y = cali_cfl.predict(X_events)
    
    # saving the model to disk for later usage
    pickle.dump(cali_cfl, open(file_name, "wb"))
    return predict_y, predict_y_proba
  else:
    # loading the model
    cali_cfl = pickle.load(open(file_name, "rb"))
    predict_y_proba = cali_cfl.predict_proba(X_events)
    predict_y = cali_cfl.predict(X_events)
    return predict_y, predict_y_proba


## Method to train and cross validate both models on training data

In [ ]:
def train_models(all_events_df):
  devices = pd.read_csv('data/gender_age_train.csv')
  y = devices['group'].values
  y_gender = devices['gender'].values
  is_events = has_events(devices, all_events_df)

  # predicting gender with devices on events data
  devices_with_events = devices[is_events == True][['device_id']]
  y_gender_with_events = y_gender[is_events == True]

  tr_devices_events_gen, te_devices_events_gen, y_tr_events_gen, y_te_events_gen = train_test_split(
    devices_with_events, y_gender_with_events, test_size=0.1, stratify=y_gender_with_events
  )
  y_tr_gen_label = get_labels(y_tr_events_gen, training=True, label='gender')
  y_te_gen_label = get_labels(y_te_events_gen, training=False, label='gender')

  y_train_gen_pred, y_train_gen_pred_proba = train_gender_predictor_with_events(tr_devices_events_gen, y_tr_gen_label, training=True)
  
  y_test_gen_pred, y_test_gen_pred_proba = train_gender_predictor_with_events(te_devices_events_gen, y_te_gen_label, training=False)
  print ("The train log loss is:", log_loss(y_tr_gen_label, y_train_gen_pred_proba))
  print ("The train log loss is:", log_loss(y_te_gen_label, y_test_gen_pred_proba))

  # predicting group on devices with events
  y_with_events = y[is_events == True]
  tr_devices_events, te_devices_events, y_tr_events, y_te_events = train_test_split(
    devices_with_events, y_with_events, test_size=0.1, stratify=y_with_events
  )
  y_tr_label = get_labels(y_tr_events, training=True, label='group')
  y_te_label = get_labels(y_te_events, training=False, label='group')

  y_train_pred, y_train_pred_proba = train_user_group_with_events(tr_devices_events, y_train_gen_pred_proba, y_tr_label, training=True)
  y_test_pred, y_test_pred_proba = train_user_group_with_events(te_devices_events, y_test_gen_pred_proba, y_te_label, training=False)
  print ("The train log loss is:", log_loss(y_tr_label, y_train_pred_proba))
  print ("The train log loss is:", log_loss(y_te_label, y_test_pred_proba))

  # predicting gender on devices without events
  devices_no_events = devices[is_events == False][['device_id']]
  y_gender_no_events = y_gender[is_events == False]

  tr_devices_no_events_gen, te_devices_no_events_gen, y_tr_no_events_gen, y_te_no_events_gen = train_test_split(
    devices_no_events, y_gender_no_events, test_size=0.1, stratify=y_gender_no_events
  )
  y_tr_no_events_gen_label = get_labels(y_tr_no_events_gen, training=True, label='gender')
  y_te_no_events_gen_label = get_labels(y_te_no_events_gen, training=False, label='gender')

  y_train_gen_pred_no_events, y_train_gen_pred_proba_no_events = train_gender_predictor_with_no_events(
      tr_devices_no_events_gen, y_tr_no_events_gen_label, training=True)
  y_test_gen_pred_no_events, y_test_gen_pred_proba_no_events = train_gender_predictor_with_no_events(
      te_devices_no_events_gen, y_te_no_events_gen_label, training=False
    )
  print ("The train log loss is:", log_loss(y_tr_no_events_gen_label, y_train_gen_pred_proba_no_events))
  print ("The test log loss is:", log_loss(y_te_no_events_gen_label, y_test_gen_pred_proba_no_events))


  # predicting gender on devices without events
  y_no_events = y[is_events == False]

  tr_devices_no_events, te_devices_no_events, y_tr_no_events, y_te_no_events = train_test_split(
    devices_no_events, y_no_events, test_size=0.1, stratify=y_no_events
  )
  y_tr_no_events_label = get_labels(y_tr_no_events, training=True, label='group')
  y_te_no_events_label = get_labels(y_te_no_events, training=False, label='group')

  y_train_pred_no_events, y_train_pred_no_events_proba = train_user_group_with_no_events(
      tr_devices_no_events, y_train_gen_pred_proba_no_events, y_tr_no_events_label, training=True)
  y_test_pred_no_events, y_test_pred_no_events_proba = train_user_group_with_no_events(
      te_devices_no_events, y_test_gen_pred_proba_no_events, y_te_no_events_label, training=False)
  print ("The train log loss is:", log_loss(y_tr_no_events_label, y_train_pred_no_events_proba))
  print ("The train log loss is:", log_loss(y_te_no_events_label, y_test_pred_no_events_proba))


  y_true = np.concatenate([y_te_label, y_te_no_events_label])
  y_pred = np.concatenate([y_test_pred_proba, y_test_pred_no_events_proba])

  print ("The overall test log loss is:", log_loss(y_true, y_pred))

In [ ]:
if __name__ == '__main__':
  all_events_df = pd.read_csv('data/events.csv')
  train_models(all_events_df)

distance travelled  (20978, 9)
data shape before removing no variance data:  (20978, 4591)
training:  True
data shape (20978, 2843)
train dataset's shape
(20978, 2843)
(20978,)
Fitting 4 folds for each of 12 candidates, totalling 48 fits
Best params  {'max_depth': 2, 'n_estimators': 400}
distance travelled  (2331, 9)
data shape before removing no variance data:  (2331, 4591)
training:  False
data shape (2331, 2843)
testing
The train log loss is: 0.4461570421436561
The train log loss is: 0.4991837028592509
distance travelled  (20978, 9)
data shape before removing no variance data:  (20978, 4591)
training:  True
data shape (20978, 2842)
Fitting 4 folds for each of 12 candidates, totalling 48 fits
Best params  {'max_depth': 2, 'n_estimators': 300}
distance travelled  (2331, 9)
data shape before removing no variance data:  (2331, 4591)
training:  False
data shape (2331, 2842)
testing part
The train log loss is: 1.759920530655525
The train log loss is: 2.1593722289610655
data shape (46202, 

# Summary

*   With same number of features and using the single model to predict user group, was giving very poor results.
*   So, we, train four different models as described below along with their corresponding log loss



In [ ]:
from tabulate import tabulate
data = [
        ['Yes', "Gender", 0.49164734136922056],
        ["Yes", "User group (used predicted gender from above model as new feature)", 2.133802776693025],
        ["No", "Gender", 0.6511259420027772],
        ["No", "User group (used predicted gender from above model as new feature)", 2.420144409974926]
      ]
print(tabulate(data, headers=["Is events data available?", "What we are predicting?", "Test log loss"]))